In [15]:
import pandas as pd
import requests
import json
from shapely.geometry import Point, Polygon
import geopandas as gpd
import numpy as np

import requests
import logging
import time

logger = logging.getLogger("root")
logger.setLevel(logging.DEBUG)

ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)

output_filename = "lat_long_pano-id_pano-url_pc6_tweede_deel.csv"

In [16]:
lat_lon_pc6 = pd.read_csv("coordinaten_PC6_CBS2016_Google.csv")

In [17]:
lat_lon_pc6 = lat_lon_pc6[["latitude", "longitude", "postcode"]]

In [18]:
lat_lon_pc6 = lat_lon_pc6[lat_lon_pc6["postcode"]!="Amsterdam"]

In [19]:
lat_lon_pc6 = lat_lon_pc6.reset_index(drop=True)

In [20]:
temp = pd.read_csv("lat_long_pano-id_pano-url_pc6.csv")

In [21]:
temp = temp.pc6.unique()

In [22]:
lat_lon_pc6 = lat_lon_pc6[~lat_lon_pc6['postcode'].isin(temp)]

In [23]:
lat_lon_pc6 = lat_lon_pc6.reset_index(drop=True)

In [24]:
lat_lon_pc6

,latitude,longitude,postcode
0,52.374047,4.904547,1011 AX
1,52.373297,4.905700,1011 AZ
2,52.373450,4.905743,1011 BA
3,52.372980,4.902825,1011 CN
4,52.376101,4.907814,1011 DG
...,...,...,...
8604,52.305863,5.015650,1109 BX
8605,52.307856,5.017980,1109 CD
8606,52.307794,5.018142,1109 CE
8607,52.305089,5.018399,1109 CH


In [25]:
crs = {'init' : 'epsg:4326'}
def dataframe_from_json_object(j, pc6):
    
    pano_list = []  
    count = len(j['_embedded']['panoramas'])
    
    for i in range(count):      
        panorama = j['_embedded']['panoramas'][i]       
        lon, lat, _ = panorama['geometry']['coordinates']
        url = panorama['_links']['equirectangular_medium']['href']    
        pano_id = panorama['pano_id']
        
        pano_list.append([lon,lat,url,pano_id, pc6])
  
    return pano_list

In [26]:
temp_pano_list = []

lat_list = lat_lon_pc6["latitude"].to_list()
lon_list = lat_lon_pc6["longitude"].to_list()

no_panorama_postcodes = []

mission = 'mission-2019'

lent = len(lon_list)

for i in range(lent):

    lat = str(lat_list[i])
    lon = str(lon_list[i])
    found_pano = False
    pc6 = lat_lon_pc6.postcode[i]
    
    for r in range(1,26):
        radius = str(r)   
        url = 'https://api.data.amsterdam.nl/panorama/panoramas/?timestamp_after=&timestamp_before=&near=' + lon + '%2C' + lat + '&radius='+ radius +'&bbox=&srid=&newest_in_range=&tags='+mission+'&page_size=1000000'
        j = requests.get(url).json()
        #j = json.loads(response.content)
        
        if len(j['_embedded']['panoramas']) >= 1:
            found_pano = True
            
        if found_pano:
            break

    if found_pano:
        temp = dataframe_from_json_object(j,pc6)
        for p in temp:
            temp_pano_list.append(p) 
   
    if found_pano == False:
        no_panorama_postcodes.append(pc6)   

    else:
        if i % 100 == 0:
            print(i, " / 17583 done")
        if len(temp_pano_list) % 200 == 0:
            pd.DataFrame(temp_pano_list, columns=['lon','lat','url','pano_id','pc6']).to_csv("{}_bak".format(output_filename))

pd.DataFrame(temp_pano_list, columns=['lon','lat','url','pano_id','pc6']).to_csv(output_filename, encoding='utf8')

700  / 17583 done
800  / 17583 done
1000  / 17583 done
1100  / 17583 done
1200  / 17583 done
1300  / 17583 done
1400  / 17583 done
1500  / 17583 done
1600  / 17583 done
1700  / 17583 done
1800  / 17583 done
1900  / 17583 done
2000  / 17583 done
2100  / 17583 done
2200  / 17583 done
2300  / 17583 done
2400  / 17583 done
2500  / 17583 done
2600  / 17583 done
2700  / 17583 done
2800  / 17583 done
2900  / 17583 done
3000  / 17583 done
3100  / 17583 done
3200  / 17583 done
3300  / 17583 done
3400  / 17583 done
3500  / 17583 done
3600  / 17583 done
3700  / 17583 done
3800  / 17583 done
3900  / 17583 done
4100  / 17583 done
4200  / 17583 done
4300  / 17583 done
4400  / 17583 done
4500  / 17583 done
4600  / 17583 done
4800  / 17583 done
4900  / 17583 done
5000  / 17583 done
5200  / 17583 done
5300  / 17583 done
5500  / 17583 done
5600  / 17583 done
5700  / 17583 done
5800  / 17583 done
5900  / 17583 done
6000  / 17583 done
6100  / 17583 done
6200  / 17583 done
6300  / 17583 done
6400  / 17583 